In [1]:
import re
import string
import time
from copy import deepcopy
from collections import OrderedDict

import editdistance
import spacy
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm

from nltk.corpus import wordnet, stopwords
from sklearn.pipeline import make_pipeline, make_union
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, log_loss, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity

/home/PiotrRybak/.pyenv/versions/3.6.1/envs/base/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
nlp = spacy.load('en')

In [3]:
def score(pipeline, df, y, score_func=log_loss, return_df=False):
    ''' Split data into train and validation, fit model and print its score '''
    
    df_train, df_test, y_train, y_test = train_test_split(df, y, random_state=42)
    model = pipeline.fit(df_train, y_train)
    
    y_pred = model.predict(df_test)
    score_train = score_func(y_train, model.predict_proba(df_train))
    score_test = score_func(y_test, model.predict_proba(df_test))
    
    print(score_train, score_test)
    
    if return_df:
        return df_test[y_pred != y_test]
    else:
        return None

In [4]:
class Levenshtein():
    ''' Copied from: https://gist.github.com/curzona/9435822 with few minor changes '''

    def get_edits(self, s1, s2, key=hash):
        '''Calculates the levenshtein distance and the edits between two strings'''
        
        try:
            rows = self.costmatrix(
                [t.lemma_ for t in s1], 
                [t.lemma_ for t in s2], 
                key,
            )
        except AttributeError:
            rows = self.costmatrix(s1, s2, key)
            
        return self.backtrace(s1, s2, rows)

    def costmatrix(self, s1, s2, key=hash):
        ''' Generate the cost matrix for the two strings '''
        rows = []

        previous_row = range(len(s2) + 1)
        rows.append(list(previous_row))

        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (key(c1) != key(c2))
                current_row.append(min(insertions, deletions, substitutions))

            previous_row = current_row
            rows.append(previous_row)

        return rows

    def backtrace(self, s1, s2, rows):
        ''' Trace back through the cost matrix to generate the list of edits '''
        i, j = len(s1), len(s2)
        edits = []

        while not (i == 0 and j == 0):
            prev_cost = rows[i][j]
            neighbors = []

            if i != 0 and j != 0:
                neighbors.append(rows[i-1][j-1])
            if i != 0:
                neighbors.append(rows[i-1][j])
            if j != 0:
                neighbors.append(rows[i][j-1])

            min_cost = min(neighbors)

            if min_cost == prev_cost:
                i, j = i-1, j-1
                edits.append({'type':'match', 'from':s1[i], 'to':s2[j]})
            elif i != 0 and j != 0 and min_cost == rows[i-1][j-1]:
                i, j = i-1, j-1
                edits.append({'type':'substitution', 'from':s1[i], 'to':s2[j]})
            elif i != 0 and min_cost == rows[i-1][j]:
                i, j = i-1, j
                edits.append({'type':'deletion', 'from':s1[i], 'to':None})
            elif j != 0 and min_cost == rows[i][j-1]:
                i, j = i, j-1
                edits.append({'type':'insertion', 'from':None, 'to':s2[j]})

        edits.reverse()

        return edits

In [13]:
class PandasSelector(BaseEstimator, TransformerMixin):
    ''' Selects given columns from DataFrame and converts them into array '''
    
    def __init__(self, selected_columns, new_col_names):
        self.selected_columns = selected_columns
        self.new_col_names = new_col_names
    
    def fit(self, df, *args):
        return self

    def transform(self, df):
        df_new = df[self.selected_columns].copy()
        df_new.rename(
            columns=dict(zip(
                self.selected_columns, 
                self.new_col_names,
            )), 
            inplace=True,
        )
        return df_new

    
class Parser(BaseEstimator, TransformerMixin):
    ''' Parse given columns with SpaCy '''
    
    def __init__(self, selected_columns, new_col_names):
        self.selected_columns = selected_columns
        self.new_col_names = new_col_names
    
    def fit(self, df, *args):
        return self

    def transform(self, df):
        out = []
        for i, row in tqdm(df.iterrows()):
            out_row = {}
            for in_col, out_col in zip(self.selected_columns, self.new_col_names):
                out_row[out_col] = nlp(row[in_col])
            out.append(out_row)
        return out
    
    
class Common(BaseEstimator, TransformerMixin):
    ''' Base class to calculate set of statistics for common tokens of two sentences '''

    stops = set(stopwords.words("english"))
    stops.update(list(string.punctuation))
    stops.add('-PRON-')
    stops.add('be')
    
    def __init__(self, use_idf=False, ngram=1):
        self.use_idf = use_idf
        self.idf = {}
        self.n_docs = 0
        self.ngram = ngram
            
    @staticmethod
    def filter_tokens(tokens):
        return NotImplemented
    
    @staticmethod
    def get_ngrams(tokens, n=1):
        out_tokens = []

        n_tokens = len(tokens)
        for i in range(n_tokens):
            if i + n <= n_tokens:
                out_tokens.append(tuple(tokens[i:(i+n)]))

        return out_tokens
    
    def get_tokens(self, tokens):
        tokens = [t for t in tokens if t.lemma_ not in self.stops]
        tokens = self.filter_tokens(tokens)
        tokens = self.get_ngrams(tokens, n=self.ngram)
        return set(tokens)
    
    def idf_sum(self, tokens):
        if self.use_idf:
            return sum([np.log(self.n_docs/(1 + self.idf.get(t, 0))) for t in tokens])
        else:
            return len(tokens)
    
    def fit(self, rows, y, *args):
        if not self.use_idf:
            return self
        
        for dupl, row in zip(y, rows):
            self.n_docs += 1
            
            tokens1 = self.get_tokens(row['s1'])
            tokens2 = self.get_tokens(row['s2'])
            
            for token in (tokens1 | tokens2):
                if token not in self.idf:
                    self.idf[token] = 0
                self.idf[token] += 1
            
        return self
    
    def transform(self, rows):
        out = []
        for row in rows:
            tokens1 = self.get_tokens(row['s1'])
            tokens2 = self.get_tokens(row['s1'])
            
            # levenstein distance
            dist = editdistance.eval(tuple(tokens1), tuple(tokens2))
            
            out.append([
                2*self.idf_sum(tokens1 & tokens2) / (0.1 + self.idf_sum(tokens1) + self.idf_sum(tokens2)),
                self.idf_sum(tokens1 & tokens2) / max(0.1, min(self.idf_sum(tokens1), self.idf_sum(tokens2))),
                self.idf_sum(tokens1 & tokens2) / max(0.1, self.idf_sum(tokens1), self.idf_sum(tokens2)),
                self.idf_sum(tokens1 & tokens2) / (0.1 + self.idf_sum(tokens1 | tokens2)),
                dist,
                dist / (1 + len(tokens1) + len(tokens2)),
                dist / max(1, len(tokens1), len(tokens2)),
            ])
            
        return np.array(out)
    
    
class CommonEnts(Common):

    @staticmethod
    def filter_tokens(tokens):
        ''' Leaves only named entities '''
        
        return [t.lemma_ for t in tokens if t.ent_type_]

    
class CommonNouns(Common):

    @staticmethod
    def filter_tokens(tokens):
        ''' Leaves only nouns '''
        
        return [t.lemma_ for t in tokens if t.pos_ == 'NOUN']

    
class CommonVerbs(Common):

    @staticmethod
    def filter_tokens(tokens):
        ''' Leaves only verbs '''
        
        return [t.lemma_ for t in tokens if t.pos_ == 'VERB']

    
class CommonAds(Common):

    @staticmethod
    def filter_tokens(tokens):
        ''' Leaves only adjectives and adverbs '''
        
        return [t.lemma_ for t in tokens if t.pos_ == 'ADJ' or t.pos_ == 'ADV']

    
class CommonTokens(Common):

    @staticmethod
    def filter_tokens(tokens):
        ''' Leaves all tokens '''
        
        return [t.lemma_ for t in tokens]

    
class ExtractCommonTokens(Common):
    ''' Extract common tokens from two sentences '''

    @staticmethod
    def filter_tokens(tokens):
        return [t.lemma_ for t in tokens]

    def get_tokens(self, tokens):
        tokens = [t for t in tokens if t.lemma_ not in self.stops]
        tokens = self.filter_tokens(tokens)
        return set(tokens)

    def transform(self, rows):
        out = []
        for row in rows:
            tokens1 = self.get_tokens(row['s1'])
            tokens2 = self.get_tokens(row['s2'])
            
            out.append(' '.join(tokens1 & tokens2))
            
        return np.array(out)

    
class ExtractDifferenceTokens(Common):
    ''' Extract symetric sum of tokens from two sentences '''

    @staticmethod
    def filter_tokens(tokens):
        return [t.lemma_ for t in tokens]

    def get_tokens(self, tokens):
        tokens = [t for t in tokens if t.lemma_ not in self.stops]
        tokens = self.filter_tokens(tokens)
        return set(tokens)

    def transform(self, rows):
        out = []
        for row in rows:
            tokens1 = self.get_tokens(row['s1'])
            tokens2 = self.get_tokens(row['s2'])
            
            out.append(' '.join(tokens1 ^ tokens2))
            
        return np.array(out)
    
    
class QuestionFreq(BaseEstimator, TransformerMixin):
    ''' Calculates how often given question was present in train set '''
    
    def __init__(self):
        self.freq = {}
    
    @staticmethod
    def lemmas(tokens):
        return ' '.join([t.lemma_ for t in tokens])
    
    def fit(self, rows, *args):
        for row in rows:
            for q in [row['s1'], row['s2']]:
                q = self.lemmas(q)
                if q not in self.freq:
                    self.freq[q] = 0
                self.freq[q] += 1
        
        return self

    def transform(self, rows):
        out = []
        
        for row in rows:
            out.append([
                self.freq.get(self.lemmas(row['s1']), 0),
                self.freq.get(self.lemmas(row['s2']), 0),
            ])
        
        return out

    
class EstimatorToTransform(BaseEstimator, TransformerMixin):
    ''' Converts Estimator to Transform, which allows for stacking Estimators '''
    
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, *args):
        self.estimator.fit(X, *args)
        return self

    def transform(self, X):
        pred = self.estimator.predict_proba(X)[:, 1]
        return pred.reshape(-1, 1)

    
class Joiner(BaseEstimator, TransformerMixin):
    ''' Joins iterable of strings'''
    
    def fit(self, rows, y, *args):
        return self

    def transform(self, rows):
        out = []
        for row in rows:
            out.append(' '.join(row))
        
        return out

class Similarity(BaseEstimator, TransformerMixin):
    ''' Base class to calculate similarity between to sentences '''
    
    def similarity(self, t1, t2):
        return NotImplemented
    
    def calc_pairwise_sim(self, s1, s2):
        sims = []
        for t1 in s1:
            for t2 in s2:
                sims.append(self.similarity(t1, t2))
        
        if len(sims) == 0:
            sims = [1.0]
        
        return np.array(sims)
    
    def fit(self, rows, *args):
        return self
    
    def transform(self, rows):
        out = []
        for row in rows:
            max_len = max(len(row['s1']), len(row['s2']))
            sims = self.calc_pairwise_sim(row['s1'], row['s2'])
            #sims = sorted(sims, reverse=True)[:min_len]
            
            out.append([
                np.min(sims),
                np.percentile(sims, 10)*max_len,
                np.mean(sims)*max_len,
                np.percentile(sims, 90)*max_len,
                np.max(sims),
                np.std(sims),
            ])
        
        return out
    
    
class GloVeSimilarity(Similarity):
    ''' Calculates similarity as cosine distance between GloVe vectors '''
    
    def __init__(self, vec_path='glove.840B.300d.filtered.txt', dim=300):
        self.vec_path = vec_path
        self.dim = dim
        self.w2v = self.load_vec()
        
    def load_vec(self):
        ''' load word vectors from http://nlp.stanford.edu/data/glove.840B.300d.zip'''
        w2v = {}

        with open(self.vec_path, 'r') as f:
            for line in f:
                ls = line.strip().split(' ')
                word, vec = ls[0], ls[1:]
                w2v[word] = np.array(vec, dtype=np.float32).reshape(1, -1)

        return w2v
    
    def similarity(self, t1, t2):
        v1 = self.w2v.get(t1.text, None)
        v2 = self.w2v.get(t2.text, None)
        
        if v1 is not None and v2 is not None:
            return cosine_similarity(v1, v2)[0][0]
        else:
            return 0.0
    
    
class WordNetSimilarity(Similarity):
    ''' Calculates similarity as Wu-Palmer metric '''
    
    def __init__(self):
        self.synsets = {}
    
    def get_synset(self, t):
        t = t.lemma_
        
        if t in self.synsets:
            return self.synsets[t]
        else:
            synsets = wordnet.synsets(t)
            if len(synsets) > 0:
                self.synsets[t] = synsets[0]
                return synsets[0]
            else:
                return None
    
    def similarity(self, t1, t2):
        synset1 = self.get_synset(t1)
        synset2 = self.get_synset(t2)
        
        if synset1 is not None and synset2 is not None:
            return wordnet.wup_similarity(synset1, synset2) or 0.0 
        else:
            return 0.0

    
class EntitySimilarity(Similarity):
    ''' Calculates similarity as identity for named entities and 1.0 otherwise '''
    
    def similarity(self, t1, t2):
        if t1.ent_type != 0 and t2.ent_type != 0:
            return 1.0 if t1.lemma_ == t2.lemma_ else 0.0
        else:
            return 1.0
        
    
class ProbSimilarity(Similarity):
    ''' This metric doesn't have sense, but it worked the best '''
    
    def similarity(self, t1, t2):
        return t1.prob*t2.prob
        #return np.mean([t1.prob, t2.prob])
        #return min(abs(t1.prob), abs(t2.prob))/max(abs(t1.prob), abs(t2.prob))

class Logger(BaseEstimator, TransformerMixin):
    ''' It prints given message '''

    def __init__(self, msg):
        self.msg = msg
    
    def fit(self, rows, *args):
        return self
    
    def transform(self, rows):
        print(time.time(), self.msg, len(rows))
        return rows

In [14]:
class Edits(BaseEstimator, TransformerMixin):
    ''' Calculates Levenshtein distance with custom cost of substitutions '''
    
    def __init__(self):
        self.lev = Levenshtein()
        self.model = None
        self.pipeline = make_pipeline(
            make_union(
                GloVeSimilarity(),
                WordNetSimilarity(),
                EntitySimilarity(),
                ProbSimilarity(),
            ),
            make_pipeline(
                make_union(
                    EstimatorToTransform(
                        RandomForestClassifier(
                            n_estimators=100, 
                            max_depth=5,
                        )),
                    EstimatorToTransform(
                        xgb.XGBClassifier(
                            objective='binary:logistic', 
                        )),
                    EstimatorToTransform(LogisticRegression()),
                ),
            ),
            LogisticRegression(),
        )
        
    def get_edits(self, s1, s2):
        return self.lev.get_edits(s1, s2)
        
    def fit(self, rows, y, *args):
        edits_X = []
        edits_y = []
        
        # make edits
        for row, current_y in zip(rows, y):
            edits = self.get_edits(row['s1'], row['s2'])
            row = {'s1':[], 's2':[]}
            for edit in edits:
                if edit['type'] != 'match':
                    from_token = edit['from']
                    to_token = edit['to']
                    
                    if from_token is not None:
                        row['s1'].append(from_token)
                        
                    if to_token is not None:
                        row['s2'].append(to_token)
                else:
                    if len(row['s1']) > 0 and len(row['s2']) > 0:
                        edits_X.append(row)
                        edits_y.append(current_y)
                    row = {'s1':[], 's2':[]}
        
        # fit pipeline
        self.model = self.pipeline.fit(edits_X, edits_y)
        
        return self

    def transform(self, rows):
        
        # make edits
        i_edits = []
        out_edits = []
        for i, row in tqdm(enumerate(rows)):
            edits = self.get_edits(row['s1'], row['s2'])
            row_edits = {'s1':[], 's2':[]}
            
            for edit in edits:
                if edit['type'] != 'match':
                    from_token = edit['from']
                    to_token = edit['to']
                    
                    if from_token is not None:
                        row_edits['s1'].append(from_token)
                        
                    if to_token is not None:
                        row_edits['s2'].append(to_token)
                else:
                    if len(row_edits['s1']) > 0 and len(row_edits['s2']) > 0:
                        out_edits.append(row_edits)
                        i_edits.append(i)  
                    row_edits = {'s1':[], 's2':[]}
        
        # predict
        pred_edits = self.model.predict_proba(out_edits)[:, 1]
        i_preds = {}
        for i, pred in zip(i_edits, pred_edits):
            if i not in i_preds:
                i_preds[i] = []
            i_preds[i].append(pred)
                
        # aggregate
        out = []
        for i, _ in enumerate(rows):
            preds = i_preds.get(i, [1.0])
            out.append([
                np.mean(preds),
                np.min(preds),
                np.max(preds),
            ])
        
        return np.array(out)

In [20]:
%%time

pipeline_freq = make_pipeline(
    Logger('freq'),
    make_union(
        QuestionFreq(),
    ),
)

pipeline_common = make_pipeline(
    Logger('common'),
    make_union(
        CommonTokens(),
        CommonEnts(),
        CommonNouns(),
        CommonVerbs(),
        CommonAds(),
    ),
)

pipeline_bow = make_pipeline(
    Logger('bow'),
    make_union(
        make_pipeline(
            ExtractCommonTokens(),
            TfidfVectorizer(min_df=10),
        ),
        make_pipeline(
            ExtractDifferenceTokens(),
            TfidfVectorizer(min_df=10),
        ),
    ),
    EstimatorToTransform(LogisticRegression()),
)

pipeline_ensemble = make_pipeline(
    make_union(
        EstimatorToTransform(
            RandomForestClassifier(
                n_estimators=100, 
                max_depth=5,
            )),
        EstimatorToTransform(
            xgb.XGBClassifier(
                objective='binary:logistic', 
            )),
        EstimatorToTransform(LogisticRegression()),
    ),
)

pipeline_edits = make_pipeline(
    Logger('edits'),
    Edits(),
)

pipeline = make_pipeline(
    Logger('selector'),
    PandasSelector(['question1', 'question2'], ['s1', 's2']),
    
    Logger('parser'),
    Parser(['s1', 's2'], ['s1', 's2']),
    
    Logger('union'),
    make_union(
        pipeline_freq,
        pipeline_common,
        pipeline_bow,
        pipeline_edits,
    ),
    Logger('train'),
    pipeline_ensemble,
    
    Logger('ensemble'),
    LogisticRegression(),
)

CPU times: user 10.9 s, sys: 260 ms, total: 11.2 s
Wall time: 10.9 s


In [21]:
# read train data
df = pd.read_csv('raw_data/train.csv').fillna('')
df.set_index('id', inplace=True)

y = np.array(df['is_duplicate'])

In [22]:
%%time
model = pipeline.fit(df, y)

59it [00:00, 587.06it/s]

1495958265.5940852 selector 404290
1495958265.665987 parser 404290


404290it [11:41, 576.54it/s]


1495958966.9207325 union 404290
1495958966.9214745 freq 404290
1495959049.480635 common 404290
1495959109.4244938 bow 404290
1495959142.5172174 edits 404290


404290it [01:16, 5275.01it/s]


1495967456.0878403 train 404290
1495967518.5555809 ensemble 404290
CPU times: user 2h 37min 10s, sys: 23.1 s, total: 2h 37min 33s
Wall time: 2h 34min 14s


In [23]:
df_test = pd.read_csv('raw_data/test.csv').fillna('')

In [24]:
%%time
pred = model.predict_proba(df_test)[:, 1]

1495967526.1804864 selector 2345796


44it [00:00, 433.37it/s]

1495967526.5571375 parser 2345796


2345796it [1:11:31, 546.61it/s]


1495971818.2335167 union 2345796
1495971818.234361 freq 2345796
1495971927.2793283 common 2345796
1495972292.733938 bow 2345796

603it [00:00, 6025.64it/s]


1495972456.6770084 edits 2345796


2345796it [06:54, 5653.89it/s]


1496002145.03077 train 2345796
1496002165.1504977 ensemble 2345796
CPU times: user 9h 36min 46s, sys: 1min 51s, total: 9h 38min 37s
Wall time: 9h 37min 19s


In [25]:
submission = pd.DataFrame(OrderedDict({
    'test_id': df_test['test_id'], 
    'is_duplicate': pred,
})).to_csv('submission.csv', index=False)